In [5]:
import os
import torch
from torch.nn.functional import normalize
import copy
import numpy as np

from phis_generator import StlGenerator
from traj_measure import BaseMeasure
from utils import from_string_to_formula, load_pickle, dump_pickle
from kernel import StlKernel

In [8]:
sampler = StlGenerator(leaf_prob=0.4)
n_vars = 3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
mu = BaseMeasure(device=device)

In [10]:
phis = sampler.bag_sample(100000, nvars=n_vars)

In [11]:
len(phis)

100000

In [12]:
string_version = list(map(str, phis))
string_version[0]

'( ( x_1 <= 0.0557 or x_1 >= 1.3788 ) and x_0 <= 0.8512 )'

In [13]:
# Salva la lista in un file txt
with open('generated-train.txt', 'w') as file:
    for item in string_version:
        file.write(item + '\n')  # Scrive ogni elemento su una nuova riga

In [1]:
from anchor_set_generation import anchorGeneration

anchorGeneration(diff_init = True, embed_dim = 1024)

'anchor_set_1024_dim'

In [2]:
import re

def rileggi_formule(file_path):
    re_read = []
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Legge tutte le righe del file

    # Itera su ogni riga del file
    for line_number, line in enumerate(lines, start=1):
        line = line.strip()  # Rimuove eventuali spazi bianchi extra

        # Se la riga contiene una formula (es. +, -, *, /, ecc.)
        if re.search(r'[\+\-\*/=<>^]', line):  # Regex per trovare operatori matematici
            re_read.append(line)

    return re_read

In [3]:
re_read = rileggi_formule('generated-train.txt')
re_read[0]

'( ( x_1 <= 0.0557 or x_1 >= 1.3788 ) and x_0 <= 0.8512 )'

In [10]:
formulae = list(map(from_string_to_formula, re_read))
formulae[0]

In [11]:
anchor_set = load_pickle('anchor_set_1024_dim.pickle')
anchor_set[0]

In [12]:
kernel = StlKernel(mu, varn=n_vars)

gram_phis = kernel.compute_bag_bag(formulae[:1024], anchor_set)

In [13]:
import pickle 

with open("embedding_tensor_1024.pickle", "wb") as f:
    pickle.dump(gram_phis, f)

In [14]:
with open("embedding_tensor_1024.pickle", "rb") as f:
    loaded_tensor = pickle.load(f)

In [16]:
import pandas as pd

dataset = []

for formula, embedding in zip(formulae[:1000], loaded_tensor):
    embedding_str = f"tensor({embedding.tolist()})"  # Usa .tolist() per convertirlo in lista
    dataset.append([formula, embedding_str])  # Aggiungi la formula e l'embedding come stringa

# 4. Creiamo un DataFrame con pandas
df = pd.DataFrame(dataset, columns=["Formula", "Embedding"])


In [17]:
df.head()

,Formula,Embedding
0,( ( x_1 <= 0.0557 or x_1 >= 1.3788 ) and x_0 <...,"tensor([0.000958795309998095, 0.00073610449908..."
1,"( ( eventually[5,9] ( eventually ( x_0 <= -0.6...","tensor([0.0021092351526021957, 0.0005350855644..."
2,"always[18,20] ( x_0 >= 1.2398 )","tensor([0.005139028653502464, 0.03429751470685..."
3,"( x_0 <= -0.0566 until[5,inf] x_1 >= 1.1386 )","tensor([0.00027192928246222436, 0.000711385742..."
4,"( eventually[17,21] ( always[9,14] ( x_2 >= -0...","tensor([0.07262212783098221, 0.025921383872628..."


In [18]:
df.to_csv('formulas_with_embeddings.csv', index=False)